# The Sellar Problem

First, we have seen quick applications of WISDEM for to run a cost and scaling model.

Then, we used OpenMDAO to study the Betz limit and find the maximum power that can be captured by an idealized rotor. There, we declared partial derivatives and provided analytical solutions to those derivatives and allowed OpenMDAO to optimize our problem.

The Betz problem used just one `ActuatorDisc` component. In real applications of WISDEM, we will need to optimize with multiple components. This tutorial will show how to assemble multiple components in OpenMDAO. From there there, our next step will return to WISDEM to assemble a wind turbine in code.

## Multidisciplinary design optimization (MDO) terminology and XDSM diagrams

MDO has some key terms and visual diagram format called XDSM to describe optimization setups. There are couple resources to learn the basics that could be helpful. The entirety of these articles are a good reference, but there are a few key sections that have information we'll use in this demo:

- [Problem formulation section of multidisciplinary design optimization on Wikipedia](https://en.wikipedia.org/wiki/Multidisciplinary_design_optimization#Problem_formulation): Read the definitions for *design variables*, *constraints*, *objectives* and *models*.

- [Lambe and Martins: Extensions to the Design Strcuture Matrix for the Description of Multidisciplinary Desgn, Analysis, and Optimation Processes](http://mdolab.engin.umich.edu/content/extensions-design-structure-matrix): Read section 2 "Terminology and Notation" for further explanation of *design variables*, *discipline analysis*, *response variables*, *target variables* and *coupling variables*. Read section 4 about XDSM diagrams that dsecribe MDO processes.